In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
from os.path import join
import yaml
import os
import sys
sys.path.append("/n/home12/binxuwang/Github/Closed-loop-visual-insilico")
import yaml
import glob
import timm
import torch
import torch as th
import torch.nn as nn
from tqdm.auto import tqdm
from os.path import join
import pickle as pkl
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from horama import maco, plot_maco
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.transforms as T
from torchvision.transforms import ToPILImage, ToTensor, Normalize, Resize
from circuit_toolkit.CNN_scorers import TorchScorer
from circuit_toolkit.GAN_utils import upconvGAN, Caffenet
from circuit_toolkit.plot_utils import to_imgrid, show_imgrid, save_imgrid, saveallforms
from circuit_toolkit.layer_hook_utils import featureFetcher_module, featureFetcher, get_module_names
from circuit_toolkit.dataset_utils import ImagePathDataset

In [4]:
acc_stim_root = r"/n/holylabs/LABS/alvarez_lab/Everyone/Accentuate_VVS/accentuation_outputs"
config_root = r"/n/holylabs/LABS/alvarez_lab/Everyone/Accentuate_VVS/accentuation_configs"
for subject_id, filename in [
    ("red_20250428-20250430", "red_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5"), 
    ("paul_20250428-20250430", "paul_20250428-20250430_vvs-encodingstimuli_z1_rw100-400.h5"), 
    # ("venus_250426-250429", "venus_250426-250429_vvs-encodingstimuli_z1_rw80-250.h5"),
    # ("three0_250426-250501", "three0_250426-250501_vvs-encodingstimuli_z1_rw80-250.h5"),
    # ("leap_250426-250501", "leap_250426-250501_vvs-encodingstimuli_z1_rw80-250.h5"),
]:
    posthoc_PCA_dir = f"/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models/{subject_id}/posthoc_model_predict_PCA_popul_unit"
    os.makedirs(posthoc_PCA_dir, exist_ok=True)
    # config_files
    # target_subfolder = glob.glob(join(acc_stim_root, f"*{subject_id}*_accentuation"))
    os.listdir(posthoc_PCA_dir)

In [5]:
os.listdir(posthoc_PCA_dir)

['df_accentuated_paul_20250428-20250430.pkl']

In [5]:
import pickle as pkl
import pandas as pd

In [ ]:
df_acc = pd.read_pickle("/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models/leap_250426-250501/posthoc_model_predict_PCA_popul_unit/df_accentuated_leap_250426-250501.pkl")
# pkl.load(open("/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models/leap_250426-250501/posthoc_model_predict_PCA_popul_unit/df_accentuated_leap_250426-250501.pkl","rb"))

,model_name,unit_id,img_id,level,score,filepath
0,AlexNet_training_seed_01,282,0,-0.521408,-0.511481,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
1,AlexNet_training_seed_01,282,0,-1.072901,-1.063394,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
2,AlexNet_training_seed_01,282,0,-1.624393,-1.614484,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
3,AlexNet_training_seed_01,282,0,-2.175886,-2.165995,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
4,AlexNet_training_seed_01,282,0,0.030085,0.039650,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
...,...,...,...,...,...,...
5772,siglip2_vitb16,81,9,1.992109,1.735329,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
5773,siglip2_vitb16,81,9,2.489602,1.705616,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
5774,siglip2_vitb16,81,9,2.489602,1.744846,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...
5775,siglip2_vitb16,81,9,2.987096,1.699476,/n/holylabs/LABS/alvarez_lab/Everyone/Accentua...


In [6]:
data = pkl.load(open("/n/holylabs/LABS/alvarez_lab/Lab/VVS_Accentuation/Encoding_models/red_20250428-20250430/posthoc_model_predict_PCA_popul_unit/posthoc_prediction_PCA_pop_unit_red_20250428-20250430_unit19_clipag_vitb32.pkl","rb"))

In [8]:
list(data)

['config',
 'df',
 'PCA_resp',
 'population_resp',
 'target_unit_resp',
 'readout_vec',
 'readout_bias',
 'cosine_sims',
 'PCA_norm']

In [9]:
data["PCA_resp"].shape

torch.Size([5500, 750])

In [11]:
data["readout_vec"].shape

torch.Size([750])

In [ ]:
config_root = r"/n/holylabs/LABS/alvarez_lab/Everyone/Accentuate_VVS/accentuation_configs"
subject_id = "red_20250428-20250430"
config_dir = join(config_root, subject_id)
config_files = sorted(glob.glob(join(config_dir, "*.yaml")))
chan_pattern = "_Ch19_"
config_pre_chan = [f for f in config_files if chan_pattern in f]
cosine_results = []
for config_file in config_pre_chan:
    config_acc = yaml.load(open(config_file, "r"), Loader=yaml.FullLoader)
    model_name = config_acc["model_name"]
    unit_id = config_acc["unit_ids"][0]
    layer_name = config_acc["layer_name"]
    print(f"Model: {model_name}, Unit: {unit_id}, Layer: {layer_name}")
    acc_split_df = df_accentuated.query("model_name == @model_name and unit_id == @unit_id")
    assert len (acc_split_df) == 110 

    predict_PCA_feature, \
        model, transforms_pipeline, fetcher, Xtransform, readout \
            = get_PCA_basis_predictor_from_config(config_file, device="cuda")
    # accentuated_dataset = ImagePathDataset(acc_split_df["filepath"], transform=transforms_pipeline)
    # accentuated_dataloader = DataLoader(accentuated_dataset, batch_size=60, shuffle=False, num_workers=10)
    acc_img_PCA_resp = get_prediction_responses(predict_PCA_feature, transforms_pipeline, 
                                        acc_split_df["filepath"].tolist(), batch_size=120, num_workers=16)

    acc_model_unit_PCA_resp = acc_img_PCA_resp.cpu().numpy()
    readout_vec = readout.weight.data[unit_id, :].cpu()
    cosine_sims = cosine_similarity(acc_model_unit_PCA_resp, readout_vec.numpy().reshape(1, -1))
    cosine_sims = cosine_sims.flatten()
    PCA_norm = np.linalg.norm(acc_model_unit_PCA_resp, axis=1)  
    acc_split_df_cos = acc_split_df.copy()
    acc_split_df_cos["cosine_similarity"] = cosine_sims
    acc_split_df_cos["PCA_norm"] = PCA_norm
    # print(f"Cosine similarities shape: {cosine_sims.shape}")
    # print(f"Mean cosine similarity: {cosine_sims.mean():.4f}")
    # print(f"Std cosine similarity: {cosine_sims.std():.4f}")
    print(f"Min cosine similarity: {cosine_sims.min():.4f}")
    print(f"Max cosine similarity: {cosine_sims.max():.4f}")
    print(f"PCA norm max {PCA_norm.max():.4f}")
    print(f"PCA norm min {PCA_norm.min():.4f}")
    cosine_results.append({"config": config_acc, 
                           "df": acc_split_df_cos.copy(), 
                           "PCA_resp": acc_model_unit_PCA_resp.copy(), 
                           "readout_vec": readout_vec.clone(), 
                           "cosine_sims": cosine_sims.copy(), 
                           "PCA_norm": PCA_norm.copy()})
